# Cube Operations

This test case is based on a user story from Chris Kent in Applied Science.  He stated that a plot combination he frequently requires is one global plot, and a plot showing an associated curve following the dimensional sum or average of the cube.

In [ ]:
import cartopy.crs as ccrs
import iris
#import iris.coord_categorisation as ccat
import iris.plot as iplt
import iris.quickplot as qplt

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import warnings
warnings.simplefilter('ignore')

In [ ]:
% matplotlib inline

# Load and Prepare the Data

For this example I am using a 4-dimensional cube.

For the global plot the data simply has to be loaded and plotted, and for the associated curve I am collapsing one dimension by averaging the data over model level number.

In [ ]:
cube = iris.load_cube(iris.sample_data_path('uk_hires.pp'), 'air_potential_temperature')
print cube

In [ ]:
mean_cube = cube.collapsed('model_level_number', iris.analysis.MEAN)
print mean_cube

# Plot the Cubes

I now have two cubes relating to the same phenomenon which I would like to visualize simultaneously.

For the original cube selected above (and the top two plots), there would be two sliders; one for time and one for model_level_number.

The collapsed cube is associated with the original cube, so the sliders for the top plots will also control the lower plot of time vs phenomenon.  This plot would be invariant with model-level-number, but the presence of one non-applicable slider for this plot should not break it.  

In [ ]:
fig1 = plt.figure(figsize=[12, 8])

ax1 = plt.subplot(1,2,1, projection=ccrs.Mercator())
ax1.coastlines()
ax1.set_title('Air Potential Temperature')
iplt.pcolormesh(cube[0, 0])
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('Temp(K)')

ax2 = plt.subplot(1,2,2, projection=ccrs.OSGB())
ax2.coastlines()
ax2.set_global()
ax2.set_title('OSGB View (for context)')
iplt.pcolormesh(cube[0, 0])

fig2 = plt.figure(figsize=[10.6, 6])
ax = plt.subplot(111)
ax.set_title('Mean over Model Level Number of Air Potential Temperature Variance with Longitude')
ax.set_xlabel('grid longitude')
ax.set_ylabel('Air Potential Temperature (K)')
iplt.plot(mean_cube[0, 0])

fig.tight_layout()
iplt.show()

# Failure Mode

Now let's break it.

The first two plots use the same unadulterated cube, so these are intrinsically linked for all sliders and dimensions.  But the template does rely on a specific dimensionality of the cube.  This is what would happen if we loaded and tried to plot a cube with a different number of dimensions:

In [ ]:
cube = iris.load_cube(iris.sample_data_path('air_temp.pp'))
print repr(cube)

In [ ]:
fig1 = plt.figure(figsize=[12, 8])

ax1 = plt.subplot(1,2,1, projection=ccrs.Mercator())
ax1.coastlines()
ax1.set_title('Air Potential Temperature')
iplt.pcolormesh(bad_cube[0, 0])
cbar = plt.colorbar(orientation='horizontal')
cbar.set_label('Temp(K)')

ax2 = plt.subplot(1,2,2, projection=ccrs.OSGB())
ax2.coastlines()
ax2.set_global()
ax2.set_title('OSGB View (for context)')
iplt.pcolormesh(cube[0, 0])

The lower cube has undergone an operation which is reliant upon its dimension name.  If we try this operation with a cube without this dimension, this would happen:

In [ ]:
mean_cube = cube.collapsed('model_level_number', iris.analysis.MEAN)
print mean_cube